## LBM: Demo Pixel Trimming

In [ ]:
import os
import sys
import time
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import psutil

# Give this notebook access to the root package
sys.path.append('../../')  # TODO: Take this out when we upload to pypi
print(sys.path[0])

import core.io
import scanreader

import zarr
import bokeh.plotting as bpl
import holoviews as hv
import panel as pn
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt

try:
    import dask.array as da
    has_dask = True
except ImportError:
    has_dask = False

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

bpl.output_notebook()
hv.notebook_extension('bokeh')

# logging
logging.basicConfig(format="{asctime} - {levelname} - [{filename} {funcName}() {lineno}] - pid {process} - {message}",
                    filename=None, 
                    level=logging.WARNING, style="{") # this shows you just errors that can harm your program
                    # level=logging.DEBUG, style="{") # this shows you general information that developers use to trakc their program 
                    # (be careful when playing movies, there will be a lot of debug messages)

# set env variables 
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"

# this session had output planes ordered differently, we need to reorder them
chan_order = np.array([ 1, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14, 15, 16, 17, 3, 18, 19, 20, 21, 22, 23, 4, 24, 25, 26, 27, 28, 29, 30])  # this is specific to our dataset
chan_order = [x-1 for x in chan_order]  # convert to 0-based indexing

In [ ]:
overwrite = False                                 # flag to re-extract tiffs with extracted data

datapath = Path('/data2/fpo/data/raw/high_res/')   # string pointing to directory containing your data
savepath = Path('/data2/fpo/data/extraction/')           # string pointing to directory containing your data
savepath.mkdir(exist_ok=True, parents=True)

htiffs = [x for x in datapath.glob('*.tif')]      # this accumulates a list of every filepath which contains a .tif file
reader = scanreader.read_scan(str(htiffs[0]), join_contiguous=True, lbm=True, x_cut=(6,6), y_cut=(17,0))  # this should take < 2s, no actual data is being read yet

In [ ]:
data_plane = reader[:, :, :, :, :]
data_plane

### Leverage zarr directory storage to compare pixel trimming

- `zarr.open` is a convenience method that handles chunking and compression for persistant storage.
- We want to keep this data as `16 bit` integers because no calculations should be done yet.

In general, we want this value to be `~1Mb` to optimize write speed. 

```python
name = '/path/to/folder'
chunksize=[300,300,1,1]
z1 = zarr.open(f'{name}', mode='w', shape=(data.shape),chunks=chunksize, dtype='int16')
```

In [19]:
data_plane = reader[:, :, :, :, :]

(583, 528)

In [ ]:
px_save_dir = savepath / "pixel_trims"
px_save_dir.mkdir(exist_ok=True, parents=True)

# create a zarr dataset with a variable number of cut pixels
x_pixels_to_cut = range(1, 6)
for i in x_pixels_to_cut:
    # check for the existence of this file before any data is read
    px_save_str = px_save_dir / f'px_{i}.zarr'
    store = zarr.DirectoryStore(px_save_str)  # save data to persistent disk storage
    
    reader = scanreader.read_scan(str(htiffs[0]), join_contiguous=True, lbm=True, x_cut=(1,i))  # this should take < 2s, no actual data is being read yet
    data = reader[0]                                         # here, data is being read
    
    z = zarr.zeros(data.shape, chunks=(data.shape[0], data.shape[1], 1, data.shape[3]), dtype='int16', store=store, overwrite=True)
    z[:] = data             # this will auto-chunk based on the specified chunks in 'open'

In [11]:
# helpful command to print the directory content. 
# place the output of the above cell, your savepath, without the quotes
!tree -L 1 /data2/fpo/data/extracted/raw.zarr

## Visualize Initial Pixel Cuts

We can see that there is a `name_N.zarr` with a variable number of X pixels trimmed on each ROI.

Using [HoloViews](https://holoviews.org/getting_started/), we can create an interactive plot to compare the different numbers of pixels cut on each ROI.

In [ ]:
@pn.cache
def get_plot(i):
    arr = zarr.open(str(px_save_dir / f'px_{i}.zarr'), mode='r')
    return hv.Image(arr[:,:,5,400]).opts(
                width=600,
                height=600,
                title=f"pixels_cut: {i}",
                cmap='gray', 
                tools=['wheel_zoom'],
            )

# Widgets
image_widget = pn.widgets.IntSlider(name="Number of cut pixels: ", value=1, start=1, end=6)
bound_plot = pn.bind(get_plot, i=image_widget) 

# Layout of widgets and plot
    pn.Row(image_widget, sizing_mode="fixed", width=500),
    bound_plot
)

# Display the layout
layout.servable()